# Introduction #

In the last lesson, we learned how to use the transfer learning technique called fine tuning to improve our classifier. The basis of this technique was making use of the hierarchy of features within the base.

In this lesson, we'll make use another property: **invariance**. We'll see how we can extend the translation invariance we learned about in Lesson 3 to other kinds of invariance which will improve our classifier even further.

# Understanding Invariance #

An idea we touched on when looking at pooling was **invariance**. Broadly, to say that a machine learning model is invariant to some property means that it will treat two examples differing only by that property as being exactly the same. We said that pooling made a network invariant to small shifts in the position of a feature. The network simply ignores that kind of difference.

To solve a classification problem means to find a model that is invariant with respect to the class labels: it should assign the same label to all images in the same class.

Giving our model translation invariance is helpful because things in the same class tend to have the same features, even if those features are shifted around. If our classifier detects a beak and feathers, our picture is almost certainly of a bird and not a fish or a horse, regardless of where those beak and features happen to be.

<!--TODO: beak and feathers-->

# Learning Invariance #

So one way to improve a model is to build an invariance into the model itself, like we did with pooling. The other way is to *teach* the model the invariance. This is what you are doing when you show the model many different images with the same class label. It learns to treat those images the same.

<!--TODO: images of the same class-->

A typical dataset, however, won't have enough examples to fully teach the model all the ways in which images from a class might be different. If you need to improve the accuracy of your model, the best thing is to collect more data. Another (much less expensive) way is to *modify* the data you already have.

The idea behind **data augmentation** is that you can teach your model an invariance by transforming your existing data in ways that images of a class in unseen data might vary. For instance, if you are classifying cars, your classifier should know that whether a car is facing left-to-right or right-to-left doesn't affect what class the car is in. If a car is a Honda in one direction, it's also a Honda in any other direction.

<!--TODO: cars in different directions -->

So, if you augment your dataset by flipping all of your images, you help your classifier to learn that this is a distinction it should ignore.

<!--TODO: data for free-->

Many kinds of transformations can be used to augment image data. Keras provides several through its `ImageDataLoader`:

- example
- example
- example

The `albumentations` is another augmentation package; it has been popular with Kagglers in competitions.

# Example - Training with Data Augmentation #

Let's see how we can use the data augmentations provided by Keras to improve the performance of the classifier from Lesson 1.

## Step 1 - Create Data Loader with Augmentations ##

As mentioned, Keras applies augmentations through its data loader. Let's use `vertical_flip`, `zoom`, `brightness`, and `rotation`.

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

# DATA_DIR = '/kaggle/input/stanford-car-dataset-by-classes-folder/car_data/car_data'
DATA_DIR = '/home/jovyan/work/kaggle/datasets/stanford-cars-keras/car_data/car_data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'test')

ds_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

BATCH_SIZE = 16
SIZE = (150, 150)

ds_train = ds_gen.flow_from_directory(directory=TRAIN_DIR,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True,
                                      target_size=SIZE,
                                      class_mode='sparse')

ds_valid = ds_gen.flow_from_directory(directory=VALID_DIR,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True,
                                      target_size=SIZE,
                                      class_mode='sparse')

Let's take a look at the effect these augmentations had.

## Step 2 - Define Model ##

In [ ]:
from tensorflow.keras.applications import VGG16

pretrained_base = VGG16(include_top=False,
                        weights='imagenet',
                        input_shape=[*SIZE, 3])

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

model = Sequential([
    pretrained_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(196, activation='softmax'),
])

## Step 3 - Train and Evaluate ##

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

history = model.fit(ds_train,
                    validation_data=ds_valid,
                    epochs=15)

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot();

# Conclusion #